In [7]:
import pandas as pd
import os
import numpy as np
import scipy as sp
import torch

In [8]:
from scipy.stats import uniform, norm
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt

In [9]:
A = 1.2
b = -0.3
sigma = 0.2
N = 100

In [10]:
X = torch.from_numpy(uniform.rvs(size=N))
X.detach()
X = X.float()

In [11]:
e = torch.from_numpy(norm.rvs(loc=0, scale=sigma, size=N)).float()

In [12]:
y = A*X + b + e

In [13]:
y.detach()
y = y.float()

In [14]:
def sigma_to_rho(sigma):
    rho = torch.log(torch.exp(sigma) - 1)
    return rho

In [15]:
def rho_to_sigma(rho):
    sigma = torch.log(1 + torch.exp(rho))
    return sigma

In [16]:
def log_q_pdf(w, theta):
    (mu_A, mu_b, rho_A, rho_b) = theta 
    
    sigma_A = rho_to_sigma(rho_A)
    qA = torch.distributions.normal.Normal(mu_A, sigma_A)
    
    sigma_b = rho_to_sigma(rho_b)
    qb = torch.distributions.normal.Normal(mu_b, sigma_b)
    
    lp = qA.log_prob(w[0]) + qb.log_prob(w[1])
    return lp

In [17]:
def log_p_pdf(w):
    prior_mu_A = torch.tensor(0.0) 
    prior_sigma_A = torch.tensor(1.0)
    pA = torch.distributions.normal.Normal(prior_mu_A, prior_sigma_A)
    
    prior_mu_b = torch.tensor(0.0) 
    prior_sigma_b = torch.tensor(1.0)
    pb = torch.distributions.normal.Normal(prior_mu_b, prior_sigma_b)
    
    lp = pA.log_prob(w[0]) + pb.log_prob(w[1])
    return lp

In [18]:
def likelihood(y,X,w):
    A = w[0]
    b = w[1]
    
    y_pred = A*X + b
    lp = torch.distributions.normal.Normal(y_pred, 0.2).log_prob(y).sum()
    return lp

In [19]:
def f(w, theta):
    v = log_q_pdf(w, theta) - log_p_pdf(w) - likelihood(y,X,w)
    return v

In [20]:
mu_A = torch.tensor(1.0)
sigma_A = torch.tensor(0.1)
rho_A = sigma_to_rho(sigma_A)
mu_b = torch.tensor(-0.4)
sigma_b = torch.tensor(0.1)
rho_b = sigma_to_rho(sigma_b)

theta = torch.tensor([mu_A, mu_b, rho_A, rho_b], requires_grad=True)

In [21]:

lst_grad_mu = []
lst_grad_rho = []
for j in range(0,5000):
    epsilon = torch.distributions.normal.Normal(torch.tensor(0.0) , torch.tensor(1.0)).sample(torch.Size([2,]))

    with torch.no_grad():
        w = theta[0:2].clone().detach() + torch.log(1 + torch.exp(theta[2:4].clone().detach()))*epsilon
    w.requires_grad = True

    v = f(w, theta)   
    v.backward()
    v = torch.tensor([])

    with torch.no_grad():
        grad_mu = w.grad + theta.grad[0:2]
        grad_rho = w.grad*epsilon/(1 + torch.exp(-theta[2:4])) + -theta.grad[2:4]


    lst_grad_mu.append(grad_mu)
    lst_grad_rho.append(grad_rho)

    theta.grad.zero_()
    w.grad.zero_()
    v.detach()
grad_mu = torch.stack(lst_grad_mu).mean(dim=0)
grad_rho = torch.stack(lst_grad_rho).mean(dim=0)
        
 



In [22]:
grad_mu

tensor([-327.6238, -497.1631])

In [23]:
grad_rho

tensor([ 7.3430, 22.6629])

In [24]:
alpha = torch.tensor(10e-5)

In [25]:
for i in range(0,10000):
    lst_grad_mu = []
    lst_grad_rho = []
    for j in range(0,10):
        epsilon = torch.distributions.normal.Normal(torch.tensor(0.0) , torch.tensor(1.0)).sample(torch.Size([2,]))

        with torch.no_grad():
            w = theta[0:2].clone().detach() + torch.log(1 + torch.exp(theta[2:4].clone().detach()))*epsilon
        w.requires_grad = True

        v = f(w, theta)   
        v.backward()
        v = torch.tensor([])

        with torch.no_grad():
            grad_mu = w.grad + theta.grad[0:2]
            grad_rho = w.grad*epsilon/(1 + torch.exp(-theta[2:4])) + -theta.grad[2:4]


        lst_grad_mu.append(grad_mu)
        lst_grad_rho.append(grad_rho)

        theta.grad.zero_()
        w.grad.zero_()
        v.detach()
    grad_mu = torch.stack(lst_grad_mu).mean(dim=0)
    grad_rho = torch.stack(lst_grad_rho).mean(dim=0)
    
    #print(grad_mu)
    #print(grad_rho)
        
    with torch.no_grad():
        theta[0:2] -= alpha*grad_mu
        theta[2:4] -= alpha*grad_rho
    
    print(theta)



tensor([ 1.0318, -0.3514, -2.2528, -2.2545], requires_grad=True)
tensor([ 1.0583, -0.3136, -2.2547, -2.2566], requires_grad=True)
tensor([ 1.0719, -0.2981, -2.2551, -2.2571], requires_grad=True)
tensor([ 1.0782, -0.2972, -2.2554, -2.2592], requires_grad=True)
tensor([ 1.0988, -0.2715, -2.2579, -2.2620], requires_grad=True)
tensor([ 1.1124, -0.2561, -2.2581, -2.2662], requires_grad=True)
tensor([ 1.1139, -0.2624, -2.2590, -2.2687], requires_grad=True)
tensor([ 1.1188, -0.2616, -2.2595, -2.2711], requires_grad=True)
tensor([ 1.1206, -0.2670, -2.2596, -2.2737], requires_grad=True)
tensor([ 1.1340, -0.2506, -2.2610, -2.2761], requires_grad=True)
tensor([ 1.1377, -0.2503, -2.2617, -2.2783], requires_grad=True)
tensor([ 1.1399, -0.2547, -2.2626, -2.2801], requires_grad=True)
tensor([ 1.1370, -0.2713, -2.2632, -2.2848], requires_grad=True)
tensor([ 1.1383, -0.2757, -2.2639, -2.2861], requires_grad=True)
tensor([ 1.1500, -0.2621, -2.2648, -2.2894], requires_grad=True)
tensor([ 1.1671, -0.2404,

tensor([ 1.3035, -0.3547, -2.3557, -2.5001], requires_grad=True)
tensor([ 1.3071, -0.3488, -2.3564, -2.5010], requires_grad=True)
tensor([ 1.3054, -0.3528, -2.3566, -2.5024], requires_grad=True)
tensor([ 1.3051, -0.3540, -2.3569, -2.5031], requires_grad=True)
tensor([ 1.3071, -0.3509, -2.3571, -2.5035], requires_grad=True)
tensor([ 1.3046, -0.3566, -2.3572, -2.5047], requires_grad=True)
tensor([ 1.3047, -0.3567, -2.3577, -2.5058], requires_grad=True)
tensor([ 1.3065, -0.3543, -2.3594, -2.5090], requires_grad=True)
tensor([ 1.3044, -0.3587, -2.3597, -2.5094], requires_grad=True)
tensor([ 1.3036, -0.3601, -2.3598, -2.5100], requires_grad=True)
tensor([ 1.3078, -0.3533, -2.3606, -2.5117], requires_grad=True)
tensor([ 1.3067, -0.3553, -2.3610, -2.5124], requires_grad=True)
tensor([ 1.3010, -0.3657, -2.3621, -2.5141], requires_grad=True)
tensor([ 1.3011, -0.3656, -2.3624, -2.5150], requires_grad=True)
tensor([ 1.3103, -0.3509, -2.3628, -2.5159], requires_grad=True)
tensor([ 1.3064, -0.3585,

tensor([ 1.3206, -0.3708, -2.4263, -2.6582], requires_grad=True)
tensor([ 1.3207, -0.3694, -2.4265, -2.6584], requires_grad=True)
tensor([ 1.3218, -0.3663, -2.4274, -2.6592], requires_grad=True)
tensor([ 1.3159, -0.3765, -2.4280, -2.6605], requires_grad=True)
tensor([ 1.3213, -0.3670, -2.4283, -2.6611], requires_grad=True)
tensor([ 1.3260, -0.3580, -2.4288, -2.6615], requires_grad=True)
tensor([ 1.3285, -0.3545, -2.4294, -2.6622], requires_grad=True)
tensor([ 1.3273, -0.3567, -2.4302, -2.6632], requires_grad=True)
tensor([ 1.3247, -0.3620, -2.4306, -2.6642], requires_grad=True)
tensor([ 1.3249, -0.3606, -2.4304, -2.6653], requires_grad=True)
tensor([ 1.3231, -0.3640, -2.4312, -2.6670], requires_grad=True)
tensor([ 1.3307, -0.3509, -2.4316, -2.6682], requires_grad=True)
tensor([ 1.3259, -0.3593, -2.4327, -2.6691], requires_grad=True)
tensor([ 1.3220, -0.3660, -2.4327, -2.6703], requires_grad=True)
tensor([ 1.3181, -0.3734, -2.4332, -2.6710], requires_grad=True)
tensor([ 1.3175, -0.3745,

tensor([ 1.3246, -0.3581, -2.4908, -2.7689], requires_grad=True)
tensor([ 1.3200, -0.3667, -2.4909, -2.7693], requires_grad=True)
tensor([ 1.3189, -0.3679, -2.4912, -2.7700], requires_grad=True)
tensor([ 1.3204, -0.3660, -2.4916, -2.7710], requires_grad=True)
tensor([ 1.3231, -0.3617, -2.4921, -2.7731], requires_grad=True)
tensor([ 1.3278, -0.3547, -2.4924, -2.7737], requires_grad=True)
tensor([ 1.3246, -0.3614, -2.4930, -2.7754], requires_grad=True)
tensor([ 1.3251, -0.3614, -2.4933, -2.7765], requires_grad=True)
tensor([ 1.3225, -0.3671, -2.4935, -2.7769], requires_grad=True)
tensor([ 1.3255, -0.3615, -2.4937, -2.7783], requires_grad=True)
tensor([ 1.3203, -0.3715, -2.4940, -2.7794], requires_grad=True)
tensor([ 1.3200, -0.3705, -2.4942, -2.7801], requires_grad=True)
tensor([ 1.3300, -0.3535, -2.4956, -2.7816], requires_grad=True)
tensor([ 1.3260, -0.3612, -2.4962, -2.7821], requires_grad=True)
tensor([ 1.3252, -0.3627, -2.4970, -2.7828], requires_grad=True)
tensor([ 1.3280, -0.3592,

tensor([ 1.3174, -0.3778, -2.5460, -2.8661], requires_grad=True)
tensor([ 1.3232, -0.3674, -2.5461, -2.8670], requires_grad=True)
tensor([ 1.3261, -0.3625, -2.5463, -2.8673], requires_grad=True)
tensor([ 1.3155, -0.3812, -2.5470, -2.8682], requires_grad=True)
tensor([ 1.3154, -0.3818, -2.5474, -2.8692], requires_grad=True)
tensor([ 1.3187, -0.3765, -2.5480, -2.8702], requires_grad=True)
tensor([ 1.3215, -0.3713, -2.5485, -2.8713], requires_grad=True)
tensor([ 1.3239, -0.3668, -2.5486, -2.8719], requires_grad=True)
tensor([ 1.3180, -0.3763, -2.5491, -2.8727], requires_grad=True)
tensor([ 1.3240, -0.3669, -2.5495, -2.8731], requires_grad=True)
tensor([ 1.3252, -0.3649, -2.5500, -2.8734], requires_grad=True)
tensor([ 1.3235, -0.3688, -2.5502, -2.8738], requires_grad=True)
tensor([ 1.3279, -0.3616, -2.5513, -2.8749], requires_grad=True)
tensor([ 1.3281, -0.3612, -2.5516, -2.8753], requires_grad=True)
tensor([ 1.3314, -0.3547, -2.5518, -2.8762], requires_grad=True)
tensor([ 1.3323, -0.3526,

...

tensor([ 1.3270, -0.3659, -3.2913, -3.8146], requires_grad=True)
tensor([ 1.3239, -0.3714, -3.2914, -3.8146], requires_grad=True)
tensor([ 1.3245, -0.3700, -3.2914, -3.8145], requires_grad=True)
tensor([ 1.3261, -0.3673, -3.2915, -3.8145], requires_grad=True)
tensor([ 1.3234, -0.3717, -3.2916, -3.8146], requires_grad=True)
tensor([ 1.3255, -0.3678, -3.2916, -3.8146], requires_grad=True)
tensor([ 1.3261, -0.3668, -3.2916, -3.8147], requires_grad=True)
tensor([ 1.3273, -0.3650, -3.2916, -3.8147], requires_grad=True)
tensor([ 1.3262, -0.3672, -3.2917, -3.8146], requires_grad=True)
tensor([ 1.3254, -0.3685, -3.2917, -3.8146], requires_grad=True)
tensor([ 1.3252, -0.3690, -3.2917, -3.8147], requires_grad=True)
tensor([ 1.3279, -0.3644, -3.2918, -3.8147], requires_grad=True)
tensor([ 1.3280, -0.3644, -3.2918, -3.8148], requires_grad=True)
tensor([ 1.3285, -0.3636, -3.2918, -3.8148], requires_grad=True)
tensor([ 1.3277, -0.3653, -3.2919, -3.8149], requires_grad=True)
tensor([ 1.3238, -0.3719,

tensor([ 1.3225, -0.3646, -3.2939, -3.8168], requires_grad=True)
tensor([ 1.3247, -0.3611, -3.2939, -3.8169], requires_grad=True)
tensor([ 1.3243, -0.3620, -3.2939, -3.8169], requires_grad=True)
tensor([ 1.3238, -0.3630, -3.2939, -3.8169], requires_grad=True)
tensor([ 1.3237, -0.3630, -3.2940, -3.8170], requires_grad=True)
tensor([ 1.3230, -0.3640, -3.2939, -3.8169], requires_grad=True)
tensor([ 1.3231, -0.3638, -3.2939, -3.8169], requires_grad=True)
tensor([ 1.3212, -0.3672, -3.2940, -3.8169], requires_grad=True)
tensor([ 1.3203, -0.3691, -3.2940, -3.8169], requires_grad=True)
tensor([ 1.3219, -0.3668, -3.2940, -3.8170], requires_grad=True)
tensor([ 1.3204, -0.3694, -3.2940, -3.8170], requires_grad=True)
tensor([ 1.3189, -0.3715, -3.2940, -3.8170], requires_grad=True)
tensor([ 1.3206, -0.3693, -3.2939, -3.8170], requires_grad=True)
tensor([ 1.3208, -0.3685, -3.2940, -3.8170], requires_grad=True)
tensor([ 1.3200, -0.3698, -3.2940, -3.8171], requires_grad=True)
tensor([ 1.3218, -0.3668,

tensor([ 1.3254, -0.3629, -3.2956, -3.8187], requires_grad=True)
tensor([ 1.3221, -0.3692, -3.2956, -3.8187], requires_grad=True)
tensor([ 1.3243, -0.3663, -3.2956, -3.8187], requires_grad=True)
tensor([ 1.3249, -0.3656, -3.2957, -3.8188], requires_grad=True)
tensor([ 1.3242, -0.3664, -3.2957, -3.8188], requires_grad=True)
tensor([ 1.3249, -0.3650, -3.2956, -3.8186], requires_grad=True)
tensor([ 1.3253, -0.3639, -3.2957, -3.8187], requires_grad=True)
tensor([ 1.3255, -0.3638, -3.2957, -3.8188], requires_grad=True)
tensor([ 1.3268, -0.3617, -3.2956, -3.8188], requires_grad=True)
tensor([ 1.3262, -0.3628, -3.2956, -3.8188], requires_grad=True)
tensor([ 1.3267, -0.3617, -3.2957, -3.8187], requires_grad=True)
tensor([ 1.3254, -0.3640, -3.2957, -3.8188], requires_grad=True)
tensor([ 1.3248, -0.3648, -3.2957, -3.8188], requires_grad=True)
tensor([ 1.3244, -0.3658, -3.2957, -3.8188], requires_grad=True)
tensor([ 1.3239, -0.3667, -3.2958, -3.8188], requires_grad=True)
tensor([ 1.3215, -0.3711,

tensor([ 1.3230, -0.3658, -3.2973, -3.8210], requires_grad=True)
tensor([ 1.3240, -0.3648, -3.2973, -3.8210], requires_grad=True)
tensor([ 1.3226, -0.3665, -3.2972, -3.8210], requires_grad=True)
tensor([ 1.3230, -0.3664, -3.2971, -3.8210], requires_grad=True)
tensor([ 1.3233, -0.3660, -3.2970, -3.8210], requires_grad=True)
tensor([ 1.3255, -0.3620, -3.2971, -3.8209], requires_grad=True)
tensor([ 1.3234, -0.3660, -3.2971, -3.8209], requires_grad=True)
tensor([ 1.3222, -0.3682, -3.2971, -3.8209], requires_grad=True)
tensor([ 1.3244, -0.3640, -3.2971, -3.8209], requires_grad=True)
tensor([ 1.3220, -0.3683, -3.2971, -3.8210], requires_grad=True)
tensor([ 1.3202, -0.3720, -3.2972, -3.8210], requires_grad=True)
tensor([ 1.3209, -0.3707, -3.2972, -3.8209], requires_grad=True)
tensor([ 1.3207, -0.3711, -3.2973, -3.8210], requires_grad=True)
tensor([ 1.3215, -0.3700, -3.2973, -3.8210], requires_grad=True)
tensor([ 1.3234, -0.3669, -3.2973, -3.8210], requires_grad=True)
tensor([ 1.3244, -0.3661,

tensor([ 1.3226, -0.3656, -3.2986, -3.8224], requires_grad=True)
tensor([ 1.3204, -0.3694, -3.2987, -3.8225], requires_grad=True)
tensor([ 1.3206, -0.3689, -3.2988, -3.8225], requires_grad=True)
tensor([ 1.3213, -0.3677, -3.2987, -3.8226], requires_grad=True)
tensor([ 1.3208, -0.3680, -3.2987, -3.8226], requires_grad=True)
tensor([ 1.3211, -0.3674, -3.2986, -3.8225], requires_grad=True)
tensor([ 1.3208, -0.3682, -3.2986, -3.8225], requires_grad=True)
tensor([ 1.3228, -0.3651, -3.2986, -3.8226], requires_grad=True)
tensor([ 1.3228, -0.3650, -3.2986, -3.8226], requires_grad=True)


In [170]:
rho_to_sigma(torch.tensor(0.5488))

tensor(1.0047)